### Dask client + PyTorch

In [1]:
from dask.distributed import Client, fire_and_forget

client = Client(memory_limit='5GB', n_workers=2, processes=True, dashboard_address=':8791')
display(client)

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8791/status,
Dashboard: http://127.0.0.1:8791/status,Workers: 2
Total threads: 8,Total memory: 9.31 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43979,Workers: 2
Dashboard: http://127.0.0.1:8791/status,Total threads: 8
Started: Just now,Total memory: 9.31 GiB
Comm: tcp://127.0.0.1:41255,Total threads: 4
Dashboard: http://127.0.0.1:38085/status,Memory: 4.66 GiB
Nanny: tcp://127.0.0.1:33803,


In [ ]:
import time
import numpy as np
import torch
N = 1000

class A:
    def __init__(self):
        self.x = torch.rand(N, N)

class Kernel:
    def __init__(self):
        self.count = 0
        
    def kernel(self, obj):
        self.count += 1
        y = torch.rand(N, N)
        return torch.matmul(obj.x, y)

op = Kernel()
for _ in range(100):
    obj = A()
    obj_ = client.scatter(obj)
    future = client.submit(op.kernel, obj_)
    fire_and_forget(future)

# op.kernel(A())
# op.count